In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
mydata=pd.read_csv('top30.csv',index_col = 0)

In [3]:
test_train = mydata[(mydata['Date']>'2010-01-14')&(mydata['Date']<'2010-09-01')].drop(columns =['Date'])
oot = mydata[mydata['Date']>='2010-09-01'].drop(columns =['Date'])

In [4]:
X= test_train.drop(['Fraud'],axis=1)
y = test_train['Fraud']
oot_x=oot.drop(['Fraud'],axis=1)
oot_y=oot['Fraud']

In [5]:
def get_result(X_train,y_train,model):
    
    result = pd.DataFrame({'predicted':model.predict_proba(X_train)[:,1],'Fraud':y_train})
    topRows = int(round(len(y_train)*0.03))
    temp = result.sort_values('predicted',ascending=False).head(topRows)
    needed=temp.loc[:,'Fraud']
    temp2 = result.sort_values('predicted',ascending=True).head(topRows)
    needed2=temp2.loc[:,'Fraud']

    return max(sum(needed),sum(needed2))/sum(y_train)
    

## Methods

In [6]:
n_estimators = [100,150,200]
n_jobs = [5,10]
max_depth = [20,30]
# max_features =[5,10,20,30]
# min_samples_split =[2,3,4,5]
# min_samples_leaf = [1,2,3,4]
result = []
for n in n_estimators:
    for nj in n_jobs:
        for m in max_depth:

            FDR3 = pd.DataFrame()
            for i in range(10):
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.40)
                scaler = StandardScaler()

                X_train_z = scaler.fit_transform(X_train)
                X_test_z = scaler.transform(X_test)
                oot_x_z = scaler.transform(oot_x)
                model = RandomForestClassifier(n_estimators=n,n_jobs =nj,max_depth=m)
                model.fit(X_train_z,y_train)

                FDR3.loc[i,'trn']=get_result(X_train_z,y_train,model)
                FDR3.loc[i,'tst']=get_result(X_test_z,y_test,model)
                FDR3.loc[i,'oot']=get_result(oot_x_z,oot_y,model)
            print(n,nj,m,FDR3.mean())
            result.append([n,nj,m,FDR3.mean()])


100 5 20 trn    1.000000
tst    0.985824
oot    0.725843
dtype: float64
100 5 30 trn    1.000000
tst    0.979617
oot    0.757865
dtype: float64
100 10 20 trn    1.000000
tst    0.988715
oot    0.789888
dtype: float64
100 10 30 trn    1.000000
tst    0.988407
oot    0.747753
dtype: float64
150 5 20 trn    1.000000
tst    0.985635
oot    0.733708
dtype: float64
150 5 30 trn    1.000000
tst    0.985296
oot    0.747753
dtype: float64
150 10 20 trn    1.000000
tst    0.981620
oot    0.730618
dtype: float64
150 10 30 trn    1.000000
tst    0.985190
oot    0.754494
dtype: float64
200 5 20 trn    1.000000
tst    0.983895
oot    0.727247
dtype: float64
200 5 30 trn    1.000000
tst    0.986524
oot    0.796067
dtype: float64
200 10 20 trn    1.000000
tst    0.983313
oot    0.780618
dtype: float64
200 10 30 trn    1.00000
tst    0.98765
oot    0.76573
dtype: float64


In [30]:
scaled = pd.DataFrame(columns = ['n_estimators','n_jobs','max_depths','3%FDR'],data=result)
scaled['Train'] =scaled['3%FDR'].apply(lambda x: x['trn'])
scaled['Test'] =scaled['3%FDR'].apply(lambda x: x['tst'])
scaled['oot'] =scaled['3%FDR'].apply(lambda x: x['oot'])
scaled.drop('3%FDR',axis = 1,inplace = True)

In [32]:
scaled

,n_estimators,n_jobs,max_depths,Train,Test,oot
0,100,5,20,1.0,0.985824,0.725843
1,100,5,30,1.0,0.979617,0.757865
2,100,10,20,1.0,0.988715,0.789888
3,100,10,30,1.0,0.988407,0.747753
4,150,5,20,1.0,0.985635,0.733708
5,150,5,30,1.0,0.985296,0.747753
6,150,10,20,1.0,0.981620,0.730618
7,150,10,30,1.0,0.985190,0.754494
8,200,5,20,1.0,0.983895,0.727247
9,200,5,30,1.0,0.986524,0.796067
